# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [8]:
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.model_selection import train_test_split
from azureml.core import Dataset
from azureml.core import Model
from azureml.core import Environment

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [9]:
myenv = Environment(name = "myenv")

In [10]:
ws = Workspace.from_config()
myenv.register(workspace=ws)

# choose a name for experiment
experiment_name = 'auto_ml_experiment'

experiment=Experiment(ws, experiment_name)


datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./cleandata.csv'],
                       target_path = 'train-dataset/tabular/',
                       overwrite = True,
                       show_progress = True)

ds = Dataset.Tabular.from_delimited_files(path = [(datastore, 'train-dataset/tabular/cleandata.csv')])

df = ds.to_pandas_dataframe()

x_col = ['Avg_Sleepiness', 'People_Queried_About_Sleep', 'raw_Crashes_per_Year','Avg_Crash_Severity','Percent_Morning_Crashes','Percent_Evening_Crashes','Crashes_per_Year']
y_col = ['State']
x_df = df.loc[:, x_col]
y_df = df.loc[:, y_col]

x_train, x_test = train_test_split(x_df, shuffle=False)
y_train, y_test = train_test_split(y_df, shuffle=False)

Uploading an estimated of 1 files
Uploading ./cleandata.csv
Uploaded ./cleandata.csv, 1 files out of an estimated total of 1
Uploaded 1 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [13]:
# TODO: Put your automl settings here
automl_settings = {}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=x_train,
    label_column_name='Crashes_per_Year',
    n_cross_validations=5
)

In [14]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
auto_ml_experiment,AutoML_ccf837c6-9237-4644-92e9-92916784c721,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected i

2023/06/06 23:35:03 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [21]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [19]:
best_model = remote_run.get_best_child()
print(best_model)

Run(Experiment: auto_ml_experiment,
Id: AutoML_ccf837c6-9237-4644-92e9-92916784c721_0,
Type: None,
Status: Completed)


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [20]:
model_name = best_model.properties['model_name']
model = best_model.register_model(model_name= model_name, model_path='outputs/model.pkl')

In [25]:
from azureml.core.environment import Environment
from azureml.automl.core.shared import constants
from azureml.core.model import InferenceConfig

best_model.download_file(constants.CONDA_ENV_FILE_PATH, 'depend.yml')
myenv = Environment.from_conda_specification(name="myenv", file_path="depend.yml")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

In [27]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model


best_model.download_file('outputs/scoring_file_v_1_0_0.py', output_file_path='score.py')

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "aciservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-06-06 23:55:11+00:00 Creating Container Registry if not exists.
2023-06-06 23:55:11+00:00 Registering the environment.
2023-06-06 23:55:13+00:00 Building image.

TODO: In the cell below, send a request to the web service you deployed to test it.

In [30]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

In [38]:
import requests
import json

# URL for the web service
scoring_uri = service.scoring_uri

# Two sets of data to score, so we get two results back
data = {
  "data": [
    {
      "Avg_Crash_Severity": 1.0,
      "Avg_Sleepiness": "36%",
      "People_Queried_About_Sleep": 1000,
      "Percent_Evening_Crashes": "50%",
      "Percent_Morning_Crashes": "50%",
      "raw_Crashes_per_Year": 1.0
    }
  ],
  "method": "predict"
}
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

"{\"result\": [0.0154494382022472]}"


TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
